In [9]:
import speech_recognition as sr
import wave
import time
from datetime import datetime

import pyaudio
import pyttsx3

import openai

import os
import re
import keyboard
from transitions import Machine

from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
FORMAT        = pyaudio.paInt16
SAMPLE_RATE   = 44100        # サンプリングレート
CHANNELS      = 1            # モノラルかバイラルか
INPUT_DEVICE_INDEX = 0       # マイクのチャンネル
CALL_BACK_FREQUENCY = 3      # コールバック呼び出しの周期[sec]


OUTPUT_TXT_FILE = "./" + datetime.now().strftime('%Y%m%d_%H_%M') +".txt" # テキストファイルのファイル名を日付のtxtファイルにする

def callback(in_data, frame_count, time_info, status):
    """
    コールバック関数の定義
    """
    
    global sprec # speech_recognitionオブジェクトを毎回作成するのではなく、使いまわすために、グローバル変数で定義しておく

    try:
        audiodata  = sr.AudioData(in_data, SAMPLE_RATE, 2)
        sprec_text = sprec.recognize_google(audiodata, language='ja-JP')
        
        with open(OUTPUT_TXT_FILE,'a') as f: #ファイルの末尾に追記していく
            f.write("\n" + sprec_text)
    
    except sr.UnknownValueError:
        pass
    
    except sr.RequestError as e:
        pass
    
    finally:
        return (None, pyaudio.paContinue)


def realtime_textise():
    """
    リアルタイムで音声を文字起こしする
    """

    with open(OUTPUT_TXT_FILE, 'w') as f: #txtファイルの新規作成
        DATE = datetime.now().strftime('%Y%m%d_%H:%M:%S')
        f.write("日時 : " + DATE + "\n") # 最初の一行目に日時を記載する

    global sprec # speech_recognitionオブジェクトを毎回作成するのではなく、使いまわすために、グローバル変数で定義しておく
    
    # speech recogniserインスタンスを生成
    sprec = sr.Recognizer() 
    
    # Audio インスタンス取得
    audio  = pyaudio.PyAudio() 
    
    # ストリームオブジェクトを作成
    stream = audio.open(format             = FORMAT,
                        rate               = SAMPLE_RATE,
                        channels           = CHANNELS,
                        input_device_index = INPUT_DEVICE_INDEX,
                        input              = True, 
                        frames_per_buffer  = SAMPLE_RATE*CALL_BACK_FREQUENCY, # CALL_BACK_FREQUENCY 秒周期でコールバック
                        stream_callback    = callback)
    
    stream.start_stream()
    
    try:
        while stream.is_active():
            time.sleep(0.1)
    except KeyboardInterrupt:
        print("end")
    
    stream.stop_stream()
    stream.close()
    audio.terminate()


def main():
    #look_for_audio_input()
    realtime_textise()


if __name__ == '__main__':
    main()

In [116]:
# Chat-GPT
API_KEY = os.getenv('OPENAI_API_KEY')
class ekimaeGPT:
    def __init__(self) -> None:
        # ChatGPTの設定
        openai.api_key = API_KEY
        self.pattern =re.compile(r'[a-z]+:')
        
        # 発話機能の設定
        self.engine = pyttsx3.init()
        self.voices = self.engine.getProperty('voices')
        self.engine.setProperty("voice", self.voices[1].id)
        
        # AudioListenerの設定
        self.FORMAT        = pyaudio.paInt16
        self.SAMPLE_RATE   = 44100        # サンプリングレート
        self.CHANNELS      = 1            # モノラルかバイラルか
        self.INPUT_DEVICE_INDEX = 3       # マイクのチャンネル
        self.CALL_BACK_FREQUENCY = 0.1
        self.REFLESH_RATE = 100 / 1000
        self.sprec = sr.Recognizer() 
        self.audio  = pyaudio.PyAudio()
        self.inputwave = []
        self.end_flag = 0

    def request2gpt(self, request_text):
        start_sequence = "\nAI:"
        restart_sequence = "\nHuman: "

        response = openai.Completion.create(
            model="text-davinci-003",
            prompt="Human: "+request_text,
            temperature=0.9,
            max_tokens=150,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0.6,
            stop=[" Human:", " AI:"]
        )

        response_text = response['choices'][0]['text'].replace('\n', '').replace('Robot: ', '')
        response_text = self.pattern.sub('', response_text)
        return response_text
    
    def speech(self, text):
        self.engine.say(text)
        self.engine.runAndWait()
    
    def talk2gpt(self, request_text):
        res = self.request2gpt(request_text)
        print(res)
        self.speech(res)
    
    def start_pyaudio(self):
        # 状態がlisteningに移ったときに実行
        # 並列で録音開始、
        self.end_flag = 1
        self.inputwave = []
        self.stream = self.audio.open(
            format=self.FORMAT,
            channels=self.CHANNELS,
            rate=self.SAMPLE_RATE,
            input_device_index = self.INPUT_DEVICE_INDEX,
            input=True,
            frames_per_buffer=int(self.SAMPLE_RATE*self.CALL_BACK_FREQUENCY),
            stream_callback=self.audio_callback
        )
        self.stream.start_stream()
        print("start rec")
        
        
    def start_process(self):
        # listeningが終了しprocessに移ったときに実行
        self.stream.stop_stream()
        self.stream.close()
        print("end rec")
        
        try:
            print("start trans")
            audiodata  = sr.AudioData(b''.join(self.inputwave), self.SAMPLE_RATE, 2)
            print(audiodata)
            sprec_text = self.sprec.recognize_google(audiodata, language='en-US')
            #sprec_text = "Hi, I'm Kenji."
            print(sprec_text)
            self.talk2gpt(sprec_text)
            #self.res()  # 状態をidleに戻す
            self.end_flag = 0
        
        except sr.UnknownValueError:
            print(sr.UnknownValueError)
            self.end_flag = 0
        
        except sr.RequestError as e:
            print(e)
            self.end_flag = 0
        
        finally:
            return (None, pyaudio.paContinue)
    
    def audio_callback(self, in_data, frame_count, time_info, status_flags):
        self.inputwave.append(in_data)
        if self.state!="listening":
            return None, pyaudio.paComplete
        return None, pyaudio.paContinue
    
    def callback(self):
        # サンプリングレート毎に実行する処理
        #print("status: " + self.state, end="")
        pass
    
    def activate(self):
        # 実行
        while self.exit_flag==0:
            self.callback()
            time.sleep(self.REFLESH_RATE)



In [123]:
states = ["idle", "listening", "process", "end"]
transitions = [
    {"trigger": "start_listen", "source": "idle", "dest": "listening", "before": "start_pyaudio"},
    {"trigger": "end_listen", "source": "listening", "dest": "process", "before": "start_process"},
    {"trigger": "res", "source": "process", "dest": "idle"},
    {"trigger": "idle2quit", "source": "idle", "dest": "end"},
    {"trigger": "process2quit", "source": "process", "dest": "end"},
]

gptkun = ekimaeGPT()
machine = Machine(model=gptkun, states=states, transitions=transitions, initial='idle', auto_transitions=False)

while gptkun.state != "end":
    if keyboard.is_pressed("q"):  # qが押されているとき
        if gptkun.state == "idle":  # idleだったら終了
            gptkun.idle2quit()
        elif gptkun.state == "process":  # processでも終了
            gptkun.process2quit()
    elif keyboard.is_pressed("space"):  # spaceが押されているとき
        if gptkun.state == "idle":  # もともとidleだったらlisteningに遷移
            gptkun.start_listen()
    else:  # 何もキーが押されていないとき
        if gptkun.state == "listening":  # listening中だったらlistening終了
            gptkun.end_listen()
        elif gptkun.end_flag == 0 and gptkun.state == "process":
            gptkun.res()
    gptkun.callback()
    time.sleep(gptkun.REFLESH_RATE)

start rec
end rec
start trans
result2:
{   'alternative': [   {   'confidence': 0.97200739,
                           'transcript': 'hi nice to meet you'},
                       {'transcript': 'hi nice to meet'}],
    'final': True}
hi nice to meet you
B Hi there! It's nice to meet you too!
start rec
end rec
start trans
result2:
{   'alternative': [   {   'confidence': 0.95891434,
                           'transcript': "what's your name"},
                       {'transcript': 'what your name'},
                       {'transcript': "what's her name"},
                       {'transcript': "what's a name"},
                       {'transcript': 'what you name'}],
    'final': True}
what's your name
B My name is Bot.
start rec
end rec
start trans
result2:
{   'alternative': [   {   'confidence': 0.97219545,
                           'transcript': 'okay can I call you'},
                       {'transcript': 'okay can I call you about'},
                       {'transcript': 'okay c